# Dair-ai Emotion Classification Challenge
Este notebook es para un reto propuesto en el semillero investigativo de modelos generativos, construyendo un modelo de clasificación del dataset "Dair-ai Emotion".

In [ ]:
%pip install torch huggingface_hub ipywidgets datasets pandas

In [ ]:
from huggingface_hub import login
login()

Tengo un bug que no termino de entender y es por qué la linear layer en vez de devolverme algo de dimensión 1 me devuelve un tensor de la misma dimensión que indexedText :c

In [29]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import pandas as pd
from datasets import load_dataset

# We download the dair-ai/emotion dataset from huggingface
trainDS = load_dataset("dair-ai/emotion", split="train")
trainDS = trainDS.with_format("torch")
validDS = load_dataset("dair-ai/emotion", split="validation")
validDS = validDS.with_format("torch")
testDS = load_dataset("dair-ai/emotion", split="test")
testDS = testDS.with_format("torch")

# I will define my embeddings from a bag of words so...
vocabulary = []
for data in trainDS:
    for word in data["text"].split():
        if word not in vocabulary:
            vocabulary.append(word)
indexedWords = {word: i for i, word in enumerate(vocabulary)}

# This is a usage of the bag of words to translate words to numbers :)
sentencesWIndex = [[indexedWords[word] for word in data["text"].split()]
                   for data in trainDS]
trainDL = DataLoader(trainDS)

# I will combine our embedding layer to a linear layer that outputs our emotion
class EmotionClassificationModel(nn.Module):
    def __init__(self, vocabularySize, embeddingDim, outputDim):
        super().__init__()
        # Now we need to transform sentencesWIndex into a dense vector representation
        # Create embedding layer (matrix)
        self.embedding = nn.Embedding(vocabularySize, embeddingDim)
        self.linear = nn.Linear(embeddingDim, outputDim)
    
    def forward(self, text):
        embedded = self.embedding(text)
        classification = self.linear(embedded)
        return classification

# In OpenAI text-embedding-3-small the embedding dimension is 1536, let's try 100
emotionModel = EmotionClassificationModel(vocabularySize=len(vocabulary), embeddingDim=100, outputDim=1)
# Define a loss function
criterion = nn.BCEWithLogitsLoss()
# Define a optimizer that updates model parameters (Adam)
optimizer = torch.optim.Adam(emotionModel.parameters())

#Let's train our model
for epoch in range(35):
    epochLoss = 0
    epochAccuracy = 0

    emotionModel.train()
    for data in trainDL:
        label = data['label']
        indexedText = [indexedWords[word] for word in data['text'][0].split()]
        optimizer.zero_grad()
        # Predictions output a Tensor(n) depending on indexedText dimension (??)
        predictions = emotionModel(torch.tensor(indexedText)).squeeze(1)
        loss = criterion(predictions, label)
        
        roundedPredictions = torch.round(torch.sigmoid(predictions))
        correct = (roundedPredictions == label).float()
        accuracy = correct.sum() / len(correct)

        loss.backward()
        optimizer.step()
        epochLoss += loss.item()
        epochAccuracy += accuracy.item()

print("Epoch %d Train: Loss: %.4f Acc: %.4f" % (epoch + 1, epochLoss / len(trainDS), 
                                                  epochAccuracy / len(trainDS)))


Using the latest cached version of the dataset since dair-ai/emotion couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'split' at C:\Users\juanb\.cache\huggingface\datasets\dair-ai___emotion\split\1.0.0\9ce63038044ae35ec1305d998d1882fcecd70ec8 (last modified on Fri Jul 19 13:23:08 2024).
Using the latest cached version of the dataset since dair-ai/emotion couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'split' at C:\Users\juanb\.cache\huggingface\datasets\dair-ai___emotion\split\1.0.0\9ce63038044ae35ec1305d998d1882fcecd70ec8 (last modified on Fri Jul 19 13:23:08 2024).
Using the latest cached version of the dataset since dair-ai/emotion couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'split' at C:\Users\juanb\.cache\huggingface\datasets\dair-ai___emotion\split\1.0.0\9ce63038044ae35ec1305d998d1882fcecd70ec8 (last modified on Fri Jul 19 13:23:08 2024).


ValueError: Target size (torch.Size([1])) must be the same as input size (torch.Size([4]))